In [1]:
import pandas as pd 
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.nn.functional import normalize
from torch.utils.data import Dataset
from torchvision import transforms
from torchvision.io import read_image
from torch.utils.data import DataLoader


In [2]:
train = pd.read_csv('archive (1)\Butterfly Identification\Training_set.csv')
test = pd.read_csv('archive (1)\Butterfly Identification\Testing_set.csv')

In [3]:
train_dir = 'archive (1)/Butterfly Identification/train'
test_dir = 'archive (1)/Butterfly Identification/test'

In [4]:
image_files = []

for i in range(len(train)):
    filename = train.iloc[i,0]
    image_files += [os.path.join(train_dir, filename)] 
    
for j in range(len(test)):
    filename = test.iloc[j,0]
    image_files += [os.path.join(test_dir,filename)]

In [5]:
image_file_list = image_files
num_total = len(image_files)
print(num_total)

9285


In [6]:
transforms.ToTensor()

ToTensor()

In [7]:
class Bf_dataset(Dataset):
    def __init__(self,image_files,transformation):
        self.image_files = image_files
        self.transform = transformation

        
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self,idx):
        image = Image.open(self.image_files[idx])
        if self.transform:
            image = self.transform(image)
        return image

In [8]:
image_resize = 64


In [9]:
transformation = transforms.Compose([
    transforms.Resize(image_resize),
    transforms.CenterCrop(image_resize),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    
])

In [10]:
mydata = Bf_dataset(image_file_list,transformation)

In [11]:
dataloader = DataLoader(mydata,batch_size=64, shuffle=True)

In [12]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(100,512,4,1,bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(512,256,4,2,bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(256,128,4,2,bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(128,64,4,2,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            
            nn.ConvTranspose2d(64,3,4,2,bias=False),
            nn.Tanh()
             
        )

    def forward(self, input):
        return self.main(input)

In [13]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [16]:
nb_gpu = 1
device = torch.device("cuda:0" if (torch.cuda.is_available() and nb_gpu > 0) else "cpu")
num_epochs = 100

# Learning rate for optimizers, same value as described in the DCGAN paper
lr = 0.0002

# Beta1 hyperparameter for Adam optimizers, same value as described in the DCGAN paper
beta1 = 0.5

In [18]:
netG = Generator().to(device)

# Handle multi-GPU if desired
if (device.type == 'cuda') and (nb_gpu > 1):
    netG = nn.DataParallel(netG, list(range(nb_gpu)))

# Apply the ``weights_init`` funb_channelstion to randomly initialize all weights
#  to ``mean=0``, ``stdev=0.02``.
netG.apply(weights_init)

# Print the model
print(netG)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), bias=False)
    (13): Tanh()
  )
)


In [20]:
class Descriminator(nn.Module):
    def __init__(self):
        super(Descriminator,self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(3,64,4,2,1,bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(64,128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
        
            
            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            
            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
        
            
            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )
        
    def forward(self, input):
        return self.main(input)